In [1]:
import time
from pprint import pprint
import numpy as np
##
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
##
import gladier.tests
from gladier.client import GladierClient as GladierBaseClient

In [2]:
class KanzusTriggers:
    def __init__(self, folder_path):
        self.observer = Observer()
        self.folder_path = folder_path

    def run(self):
        print("Kanzus Triggers Started")

        event_handler = Handler()
        self.observer.schedule(event_handler, self.folder_path, recursive = True)
        self.observer.start()
        try:
            while True:
                time.sleep(1)
        except:
            self.observer.stop()
            print("Kanzus Triggers Stopped")
  
        self.observer.join()

class Handler(FileSystemEventHandler):
    @staticmethod
    def on_any_event(event):
        if event.is_directory:
            return None
        elif event.event_type == 'created':

            #event.src_path is the file watchdog found
            # Event is created, you can process it now
            KanzusLogic(event.src_path,f_pattern=None,f_ext=None, n_batch=256)
  

def KanzusLogic(cbf_file,f_pattern=None,f_ext=None, n_batch=256):
    if not f_pattern:
        f_pattern = r'(\w+)_(\d+)_\d+.cbf'
    cbf_parse = re.compile(f_pattern)
    cbf_parse(cbf_file)
    print(cbf_file, cbf_parse)

        ##cbf is created
        ##n_batch runs plot
        ##cbf name  %n_batch==0
        ## 


# exp = KanzusTriggers('.')
# exp.run()

In [3]:
from gladier_kanzus.flows.phils_beta_raf import flow_definition as flow_phil

class KanzusPhilsClient(GladierBaseClient):
    client_id = 'e6c75d97-532a-4c88-b031-8584a319fa3e'

    gladier_tools = [
        'gladier_kanzus.tools.CreatePhil',
        'gladier_kanzus.tools.DialsStills',
    ]
    flow_definition = flow_phil


phils_client = KanzusPhilsClient()

[DEBUG] gladier.client::__init__() No authorizers provided, loading from disk.


In [4]:

from gladier_kanzus.flows.gather_plot_ingest_flow import flow_definition as flow_gpi

class SSXPlotAndPublish(GladierBaseClient):
    client_id = 'e6c75d97-532a-4c88-b031-8584a319fa3e'
    gladier_tools = [
        'gladier_kanzus.tools.SSXGatherData',
        'gladier_kanzus.tools.SSXPlot',
        'gladier_kanzus.tools.SSXPublish',
    ]
    flow_definition = flow_gpi


##nick main executor
# flow_input = {
#     'input': {
#         'trigger_name': '/eagle/APSDataAnalysis/SSX/S12/PDL1/B/Bounce_7_37400.cbf',

#         # NEEDED FOR NICK
#         'funcx_endpoint_non_compute': '0a162a09-8bd9-4dd9-b046-0bfd635d38a7',
#         'funcx_endpoint_compute': '37e6099f-e9e7-4817-ac68-4afcd78d8221',
#     }
# }
# re_cli = SSXPlotAndPublish()
# corr_flow = re_cli.start_flow(flow_input=flow_input)
# action_id = corr_flow['action_id']
# re_cli.progress(action_id)
# pprint(re_cli.get_status(action_id))

In [5]:
##hacking over container
from funcx.sdk.client import FuncXClient
fxc = FuncXClient()
from gladier_kanzus.tools.dials_stills import funcx_stills_process as stills_cont
cont_dir =  '/home/rvescovi/.funcx/containers/'
container_name = "dials_v1.simg"
dials_cont_id = fxc.register_container(location=cont_dir+'/'+container_name,container_type='singularity')
stills_cont_fxid = fxc.register_function(stills_cont, container_uuid=dials_cont_id)
##

In [8]:
def create_ranges(start,end,delta):

    s_vec = np.arange(start,end,delta)
    proc_range = []
    for k in s_vec:
        k_start = k+1
        k_end = k_start + delta - 1
        if k_end > end:
            k_end = end
        proc_range.append(f'{{{str(k_start).zfill(5)}..{str(k_end).zfill(5)}}}')
    return proc_range

In [12]:
##theta
conf = {'local_endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
        'queue_endpoint': '23519765-ef2e-4df2-b125-e99de9154611',
        }
##cooley
# conf = {'local_endpoint': '83e95e2e-fd70-45ea-9467-5efe5d95ff11',
#         'queue_endpoint': 'd26622fb-3bef-44df-8874-fcfdfbcc29fd',
#         }

data_dir = '/eagle/APSDataAnalysis/SSX/S12/StTrpAB/B'
proc_dir = data_dir
proc_range_start = 0
proc_range_ends = 10109
proc_range_delta = 256

for p_range in create_ranges(proc_range_start,proc_range_ends,proc_range_delta):
    flow_input = {
        "input": {
            #Processing variables
            "proc_dir": proc_dir,
            "data_dir": data_dir,

            #Dials specific variables.
            "input_files": f"Bounce_8_{p_range}.cbf", 
            "input_range": p_range[1:-1],
            "nproc": 16,
            "beamx": "-214.400",
            "beamy": "218.200",

            # funcX endpoints
            "funcx_local_ep": conf['local_endpoint'],
            "funcx_queue_ep": conf['queue_endpoint'],

            # container hack for stills
            "stills_cont_fxid": stills_cont_fxid
        }
    }
    #pprint(flow_input['input'])
    phils_flow = phils_client.start_flow(flow_input=flow_input)


[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.CreatePhil object at 0x000001F607D743D0>
[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.DialsStills object at 0x000001F607D74A90>
[INFO] gladier.client::start_flow() Started flow kanzus_phils_client flow id "d2df4552-1658-4b25-b73e-6c2c40a8eb16" with action "70898400-e321-4f26-b56b-748e55bb50d0"
[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.CreatePhil object at 0x000001F607BF0C40>
[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.DialsStills object at 0x000001F607BF0C10>
[INFO] gladier.client::start_flow() Started flow kanzus_phils_client flow id "d2df4552-1658-4b25-b73e-6c2c40a8eb16" with action "ace74843-7910-4d54-8f9f-d86725b70bf4"
[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.CreatePhil object at 0x000001F